In [2]:
import os
import numpy as np
import pandas as pd
import time
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    matthews_corrcoef, f1_score, precision_score, recall_score,
    accuracy_score, roc_auc_score, average_precision_score
)

from pyod.models.ae1svm import AE1SVM
from pyod.models.devnet import DevNet
from pyod.models.lunar import LUNAR
from pyod.models.cblof import CBLOF
from pyod.models.knn import KNN
from pyod.models.iforest import IForest
from pyod.models.ocsvm import OCSVM
from pyod.models.lof import LOF
from pyod.models.deep_svdd import DeepSVDD
from pyod.models.hbos import HBOS
from pyod.models.pca import PCA
from pyod.models.sod import SOD
from pyod.models.cof import COF
from pyod.models.loda import LODA
from pyod.models.alad import ALAD
from pyod.models.ecod import ECOD
from pyod.models.copod import COPOD
from pyod.models.auto_encoder import AutoEncoder
from pyod.models.so_gaal import SO_GAAL  # đúng: "sogaal" (không phải "so_gaal")
from pyod.models.mo_gaal import MO_GAAL  # đúng: "mogaal" (không phải "mo_gaal")
from pyod.models.vae import VAE
from pyod.models.suod import SUOD

import sys
import os
sys.path.append("../..")

from baseline_model.dasvdd_wrapper import DASVDD
from baseline_model.neutralad_wrapper import NeuTraLAD
from baseline_model.dif_wrapper import DIF


models_list = [
    # "CBLOF",
    # "KNN",
    # "IForest",
    # "OCSVM",
    # "LOF",
    # "DeepSVDD",
    # "HBOS",
    # "PCA",
    # "LODA",
    # "ECOD",
    # "COPOD",
    # "AutoEncoder",
    # "DevNet",
    # "AE1SVM",
    # "ALAD",
    # "SO_GAAL",
    # "MO_GAAL" # ,
    # # "VAE"
    "LUNAR",
    "DASVDD", 
    "SUOD", 
    "NeuTraLAD",     
    "DIF", 

]

output_file = f"../../Results/additionalBaselineOutlier.csv"
columns = ["Dataset", "Model", "outlier_mode", "scaler", "AUCROC", "AUCPR", "Accuracy", "MCC", "F1 Score",
           "Precision", "Recall", "Time Train", "Time Test"]

def preprocess_data_noise(train_data, test_data, noise_percentage=0):
    """
    Preprocess the training and testing data by separating features and labels,
    and return the preprocessed feature sets and labels with added noise.
    
    Args:
        train_data (DataFrame): Training data containing features and labels.
        test_data (DataFrame): Testing data containing features and labels.
        noise_percentage (float): The percentage of training samples to add noise to.
        
    Returns:
        X_train (ndarray): Preprocessed training features with added noise.
        y_train (ndarray): Preprocessed training labels with added noise.
        X_test (ndarray): Preprocessed testing features.
        y_test (ndarray): Preprocessed testing labels.
    """
    print("..............................Data Overview................................")
    print("Train Data Shape:", train_data.shape)
    print("Test Data Shape:", test_data.shape)
    
    # Convert to numpy arrays for easier manipulation
    X_train_total = train_data.iloc[:, :-1].to_numpy()
    y_train_total = train_data.iloc[:, -1].to_numpy()

    # Separate the samples with label 0
    X_train = X_train_total[y_train_total == 0]
    y_train = y_train_total[y_train_total == 0]

    print("Train Data Labels [0]:", np.unique(y_train))

    # Calculate how many samples to add noise to based on the provided percentage
    n_samples = X_train.shape[0]
    noise_samples_count = int(n_samples * (noise_percentage / 100))

    # Get the samples with label 1 (for generating noise)
    X_train_noise = X_train_total[y_train_total == 1]
    
    # Randomly select noise_samples_count from X_train_noise
    noisy_indices = np.random.choice(X_train_noise.shape[0], size=noise_samples_count, replace=False)
    X_train_noise = X_train_noise[noisy_indices]
    
    # Add the noisy samples to the training set
    X_train = np.vstack((X_train, X_train_noise))
    # y_train = np.concatenate((y_train, np.ones(X_train_noise.shape[0])))
    y_train = np.concatenate((y_train, np.zeros(X_train_noise.shape[0])))

    print(y_train) 
    # Prepare test data
    X_test = test_data.iloc[:, :-1].to_numpy()
    y_test = test_data.iloc[:, -1].to_numpy()

    # Print the new size of training data
    n_samples = X_train.shape[0]
    n_features = X_train.shape[1]
    print("Number of samples after adding noise:", n_samples)
    print("Number of features:", n_features)

    return X_train, y_train, X_test, y_test



def evaluate_model(y_true, y_pred, y_scores=None, y_probabilities=None):
    """
    Evaluates the model using multiple metrics and prints the results.
    This version includes AUCROC and AUCPR using predicted probabilities.

    Parameters:
    - y_true (np.ndarray): True labels of the test data.
    - y_pred (np.ndarray): Predicted labels of the test data.
    - y_scores (np.ndarray, optional): Scores used to compute AUCROC and AUCPR.
    - y_probabilities (np.ndarray, optional): Predicted probabilities for each class.

    Returns:
    - metrics (list): A list containing AUCROC, AUCPR, accuracy, MCC, F1 score, precision, and recall.
    """
    print("..............................Evaluation Metrics...............................")

    # Calculate standard metrics
    mcc = matthews_corrcoef(y_true, y_pred)  # Matthew's correlation coefficient
    f1 = f1_score(y_true, y_pred)  # F1 score
    precision = precision_score(y_true, y_pred)  # Precision
    recall = recall_score(y_true, y_pred)  # Recall
    accuracy = accuracy_score(y_true, y_pred)  # Accuracy

    # ROC and PR curve scores using predicted probabilities
    auc_roc, auc_pr = None, None
    if y_probabilities is not None:
        auc_roc = roc_auc_score(y_true, y_probabilities[:, 1])  # Probabilities for class 1 (outliers)
        auc_pr = average_precision_score(y_true, y_probabilities[:, 1])  # AUCPR for class 1

    # Display metrics
    print(f"AUCROC: {auc_roc * 100 if auc_roc else 'N/A'}")
    print(f"AUCPR: {auc_pr * 100 if auc_pr else 'N/A'}")
    print(f"Accuracy: {accuracy * 100:.2f}")
    print(f"MCC: {mcc:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")

    # Return the evaluation metrics as a list
    return [auc_roc * 100 if auc_roc else None, auc_pr * 100 if auc_pr else None,
            accuracy * 100, mcc, f1, precision, recall]


def get_model(name, **kwargs):
    """
    Returns the appropriate PyOD model based on the provided name.
    
    Parameters:
    - name (str): The name of the anomaly detection model (e.g., 'CBLOF', 'KNN').
    - kwargs (dict): Additional parameters to initialize the model.

    Returns:
    - model (object): An instance of the specified anomaly detection model.
    """
    model_dict = {
        "CBLOF": CBLOF,
        "KNN": KNN,
        "IForest": IForest,
        "OCSVM": OCSVM,
        "LOF": LOF,
        "DeepSVDD": DeepSVDD,
        "HBOS": HBOS,
        "SOD": SOD,
        "COF": COF,
        "LODA": LODA,
        "PCA": PCA,
        "ECOD": ECOD,
        "COPOD": COPOD,
        "AutoEncoder": AutoEncoder,
        "DevNet": DevNet,
        "LUNAR": LUNAR,
        "AE1SVM": AE1SVM,
        "ALAD": ALAD,
        "SO_GAAL": SO_GAAL,
        "MO_GAAL": MO_GAAL,
        "VAE":VAE, 
        "DASVDD": DASVDD, 
        "SUOD": SUOD, 
        "NeuTraLAD": NeuTraLAD,
        "DIF": DIF, 
    }
    
    # Fetch the model class based on the provided model name
    model_class = model_dict.get(name)
    
    # Raise an error if the model name is not found
    if model_class is None:
        raise ValueError(f"Model {name} not found.")
    
    # Return the model initialized with the provided parameters
    return model_class(**kwargs)


# Initialize output CSV file
if not os.path.exists(output_file):
    pd.DataFrame(columns=columns).to_csv(output_file, index=False)

# Define a function to run the model training and evaluation
def run_experiment(X_train, y_train, X_test, y_test, name, noise_percentage, outlier_mode, scaler ):
    for model_name in models_list:
        try:
            print(f"\nRunning dataset {name} with model {model_name}")
            
            # train_data = pd.concat([X_train, y_train], axis=1, ignore_index=True)
            # test_data = pd.concat([X_test, y_test], axis=1, ignore_index=True)

            # # Preprocess data (assuming preprocess_data is defined elsewhere)
            # X_train, y_train, X_test, y_test = preprocess_data(train_data, test_data)
            
            if model_name == 'DeepSVDD':
                start_time = time.time()
                n_features = X_train.shape[1]
                model = DeepSVDD(n_features=n_features)
                model.fit(X_train)
            elif model_name == 'DASVDD':
                start_time = time.time()
                model = DASVDD(
                    code_size=32,
                    num_epochs=100,
                    batch_size=128,
                    lr=1e-3,
                    K=0.9,
                    T=10,
                    verbose=1
                )
                model.fit(X_train)
            elif model_name == "DIF": 
                start_time = time.time()
                model = DIF(n_ensemble=50, n_estimators=6)

                # Hoặc với custom config
                model = DIF(
                    n_ensemble=50,
                    n_estimators=6,
                    max_samples=256,
                    hidden_dim=[500, 100],
                    rep_dim=20,
                    activation='tanh',
                    batch_size=64,
                    device='cuda',
                    verbose=1
                )
                model.fit(X_train)
                
            elif model_name == "NeuTraLAD": 
                start_time = time.time()
                model = NeuTraLAD(
                    latent_dim=32,      # Kích thước latent space
                    enc_hdim=32,        # Hidden dim encoder
                    num_trans=11,       # Số transformations
                    num_epochs=200,     # Số epochs
                    batch_size=128,
                    lr=0.001,
                    device='cuda',      # hoặc 'cpu'
                    verbose=1           # 1 để hiển thị progress
                )
                # Fit model (chỉ dùng normal data)
                model.fit(X_train)
            elif model_name == "SUOD": 
                start_time = time.time()
                
                base_estimators = [
                    HBOS(n_bins=10),
                    COPOD(),
                    ECOD(),
                    IForest(n_estimators=50, max_samples=128, random_state=42),
                    PCA(n_components=10, random_state=42),
                ]
                
                model = SUOD(
                    base_estimators=base_estimators,
                    n_jobs=1,                    # BẮT BUỘC = 1 để tránh hoàn toàn bug joblib
                    rp_flag_global=True,         # vẫn tăng tốc
                    bps_flag=False,              # tắt BPS → nguyên nhân chính của lỗi
                    contamination=0.1,
                    verbose=True                 # in chi tiết để bạn thấy nó chạy
                )
                
                model.fit(X_train)  # fit all models with X
            else:
                model = get_model(model_name)
                start_time = time.time()
                if model_name == 'DevNet':
                    model.fit(X_train, y_train)
                else:
                    model.fit(X_train)
                    
            train_time = time.time() - start_time

           #  columns = ["Dataset", "Model", "outlier_mode", "scaler", "AUCROC", "AUCPR", "Accuracy", "MCC", "F1 Score",
           # "Precision", "Recall", "Time Train", "Time Test"]
            # Test the model
            start_time = time.time()
            y_pred = model.predict(X_test)
            # Check if the model supports predict_proba and calculate probabilities
            if hasattr(model, "predict_proba"):
                y_probabilities = model.predict_proba(X_test)  # Class probabilities for each instance
            else:
                y_probabilities = None  # Some models do not have predict_proba method

            test_time = time.time() - start_time

            # Evaluate the model using the appropriate probabilities
            metrics = evaluate_model(y_test, y_pred, y_scores=None, y_probabilities=y_probabilities)
            result = [name, model_name] + [outlier_mode, scaler]+ metrics + [train_time, test_time]
            result_df = pd.DataFrame([result], columns=columns)
            result_df.to_csv(output_file, mode='a', header=False, index=False)

            print(f"Results saved for {name} with model {model_name}")

        except Exception as e:
            print(f"Error with dataset {name}, model {model_name}: {e}")



if __name__ == "__main__": 
    
    dataset_prefixes =  ['data_ToNIoT.csv', 'data_CICIoT2023.csv','data_N_BaIoT.csv', 'data_BoTIoT.csv']
    scaler_names = ['QuantileTransformer','MinMaxScaler','Normalizer','RobustScaler', 'StandardScaler' ]
    
    outlier_modes = [ 'local', 'cluster', 'global']

    for scaler in scaler_names: 
    
        for prefix in dataset_prefixes:
            
            for outlier_mode in outlier_modes: 
             
                # Construct file paths for train and test datasets with 'Train_' and 'Test_' prefixes
                train_file = f'../../Datascaled/OutlierData/Train_{outlier_mode}_{scaler}_{prefix}'
                test_file = f'../../Datascaled/OutlierData/Test_{outlier_mode}_{scaler}_{prefix}'
                
                if not os.path.exists(train_file):
                    print(f"❌ File không tồn tại: {train_file}, bỏ qua.")
                    continue  # Bỏ qua vòng lặp nếu file không tồn tại

                if not os.path.exists(test_file):
                    print(f"❌ File không tồn tại: {test_file}, bỏ qua.")
                    continue  # Bỏ qua vòng lặp nếu file không tồn tại
                
                # Load the CSV files
                df_train = pd.read_csv(train_file)
                df_test = pd.read_csv(test_file)
        
                
                X_train, y_train, X_test, y_test = preprocess_data_noise(df_train, df_test)
                # Run the experiments for each dataset typ
                run_experiment(X_train, y_train, X_test, y_test, prefix, 0, outlier_mode, scaler ) 
               



..............................Data Overview................................
Train Data Shape: (3000, 29)
Test Data Shape: (7000, 29)
Train Data Labels [0]: [0]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.9s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


Balanced Scheduling Total Train Time: 1.8855676651000977
Split among workers default: [5] []
Split among workers default: [5] []
Parallel score prediction...


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.9s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished


Parallel Score Prediction without Approximators Total Time: 0.8559679985046387
Split among workers default: [5] []
Parallel score prediction...
Parallel Score Prediction without Approximators Total Time: 0.06535077095031738
..............................Evaluation Metrics...............................
AUCROC: 76.89924577155108
AUCPR: 95.41309758857825
Accuracy: 26.61
MCC: 0.0715
F1 Score: 0.3139
Precision: 0.9385
Recall: 0.1885
Results saved for data_ToNIoT.csv with model SUOD

Running dataset data_ToNIoT.csv with model NeuTraLAD
Epoch 10/200, Loss: 0.464103
Epoch 20/200, Loss: 0.270389
Epoch 30/200, Loss: 0.198530
Epoch 40/200, Loss: 0.167646
Epoch 50/200, Loss: 0.154831
Epoch 60/200, Loss: 0.149075
Epoch 70/200, Loss: 0.145451
Epoch 80/200, Loss: 0.142993
Epoch 90/200, Loss: 0.141279
Epoch 100/200, Loss: 0.140064
Epoch 110/200, Loss: 0.139115
Epoch 120/200, Loss: 0.138437
Epoch 130/200, Loss: 0.137911
Epoch 140/200, Loss: 0.137470
Epoch 150/200, Loss: 0.137130
Epoch 160/200, Loss: 0

...

Parallel Score Prediction without Approximators Total Time: 0.2541775703430176
..............................Evaluation Metrics...............................
AUCROC: 99.9681630054123
AUCPR: 99.44598337950139
Accuracy: 87.71
MCC: 0.6267
F1 Score: 0.6254
Precision: 0.4550
Recall: 1.0000
Results saved for data_N_BaIoT.csv with model SUOD

Running dataset data_N_BaIoT.csv with model NeuTraLAD
Epoch 10/200, Loss: 0.484744
Epoch 20/200, Loss: 0.181777
Epoch 30/200, Loss: 0.144878
Epoch 40/200, Loss: 0.139570
Epoch 50/200, Loss: 0.137726
Epoch 60/200, Loss: 0.136768
Epoch 70/200, Loss: 0.136176
Epoch 80/200, Loss: 0.135764
Epoch 90/200, Loss: 0.135477
Epoch 100/200, Loss: 0.135255
Epoch 110/200, Loss: 0.135098
Epoch 120/200, Loss: 0.134964
Epoch 130/200, Loss: 0.134863
Epoch 140/200, Loss: 0.134766
Epoch 150/200, Loss: 0.134700
Epoch 160/200, Loss: 0.134641
Epoch 170/200, Loss: 0.134584
Epoch 180/200, Loss: 0.134540
Epoch 190/200, Loss: 0.134501
Epoch 200/200, Loss: 0.134465
................

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished


Epoch 10/200, Loss: 0.524033
Epoch 20/200, Loss: 0.313849
Epoch 30/200, Loss: 0.232547
Epoch 40/200, Loss: 0.198756
Epoch 50/200, Loss: 0.178809
Epoch 60/200, Loss: 0.166243
Epoch 70/200, Loss: 0.158802
Epoch 80/200, Loss: 0.153499
Epoch 90/200, Loss: 0.149906
Epoch 100/200, Loss: 0.147353
Epoch 110/200, Loss: 0.145252
Epoch 120/200, Loss: 0.143671
Epoch 130/200, Loss: 0.142503
Epoch 140/200, Loss: 0.141419
Epoch 150/200, Loss: 0.140604
Epoch 160/200, Loss: 0.139907
Epoch 170/200, Loss: 0.139357
Epoch 180/200, Loss: 0.138923
Epoch 190/200, Loss: 0.138521
Epoch 200/200, Loss: 0.138161
..............................Evaluation Metrics...............................
AUCROC: 78.29862500675846
AUCPR: 23.75368536882624
Accuracy: 57.41
MCC: 0.2522
F1 Score: 0.2850
Precision: 0.1697
Recall: 0.8879
Results saved for data_BoTIoT.csv with model NeuTraLAD

Running dataset data_BoTIoT.csv with model DIF
network additional parameters: {'n_hidden': [500, 100], 'n_emb': 20, 'skip_connection': None, 'dr

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished


Epoch 10/200, Loss: 0.604247
Epoch 20/200, Loss: 0.375606
Epoch 30/200, Loss: 0.258553
Epoch 40/200, Loss: 0.198148
Epoch 50/200, Loss: 0.170060
Epoch 60/200, Loss: 0.157015
Epoch 70/200, Loss: 0.150284
Epoch 80/200, Loss: 0.146224
Epoch 90/200, Loss: 0.143604
Epoch 100/200, Loss: 0.141977
Epoch 110/200, Loss: 0.140687
Epoch 120/200, Loss: 0.139700
Epoch 130/200, Loss: 0.139027
Epoch 140/200, Loss: 0.138491
Epoch 150/200, Loss: 0.137960
Epoch 160/200, Loss: 0.137610
Epoch 170/200, Loss: 0.137255
Epoch 180/200, Loss: 0.136975
Epoch 190/200, Loss: 0.136719
Epoch 200/200, Loss: 0.136502
..............................Evaluation Metrics...............................
AUCROC: 94.90409406184048
AUCPR: 49.17546715720024
Accuracy: 59.27
MCC: 0.3197
F1 Score: 0.3128
Precision: 0.1854
Recall: 1.0000
Results saved for data_BoTIoT.csv with model NeuTraLAD

Running dataset data_BoTIoT.csv with model DIF
network additional parameters: {'n_hidden': [500, 100], 'n_emb': 20, 'skip_connection': None, 'dr